# Inference for Zero DIDCE for Images

In [3]:
import os
import sys
import cv2
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import argparse

# Clear sys.argv so extra arguments from the kernel don't interfere with argparse
sys.argv = ['']

# ---------------------------
# Model Definition and Enhancement Function
# ---------------------------
class DiDCENet(nn.Module):
    def __init__(self):
        super(DiDCENet, self).__init__()
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.dropout = nn.Dropout(0.5)
        self.skip_conv = nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        x1 = self.relu(self.conv1(x))
        x2 = self.relu(self.conv2(x1))
        x3 = self.relu(self.conv3(x2))

        inverted_x = 1 - x
        x11 = self.relu(self.conv1(inverted_x))
        x21 = self.relu(self.conv2(x11))
        x31 = self.relu(self.conv3(x21))

        combined_features = torch.cat([x3, x31], dim=1)
        combined_features = self.dropout(combined_features)
        curve_map = torch.tanh(self.skip_conv(combined_features))
        return curve_map

def adaptive_light_enhancement(x_input, curve_map):
    alpha = 0.63  # Target light level
    mean_light_level = torch.mean(x_input).item()
    beta = -0.79 * mean_light_level**2 + 0.81 * mean_light_level + 1.41

    iterations_formula = (
        -25 * mean_light_level + 10 if mean_light_level < 0.1 else
        (17.14 * mean_light_level**2 - 15.14 * mean_light_level + 10 if mean_light_level < 0.45 else
         5.66 * mean_light_level**2 - 2.93 * mean_light_level + 7.2)
    )
    iterations = int(iterations_formula)

    enhanced_image = x_input.clone()  # Clone to avoid modifying original input
    for _ in range(iterations):
        enhanced_image = enhanced_image + curve_map * (enhanced_image**2 - enhanced_image) * \
                         ((alpha - torch.mean(enhanced_image).item()) / (beta - torch.mean(enhanced_image).item()))
    return enhanced_image

# ---------------------------
# Preprocessing, I/O, and Display Utilities
# ---------------------------
# Use a fixed transform to resize the image to 224x224 for the model
def get_inference_transforms():
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    return transform

# Modified loader returns both the original PIL image and the transformed tensor.
def load_image(image_path, transform):
    # Read the image using cv2 to get the original dimensions
    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        raise ValueError(f"Unable to read image at {image_path}")
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    original_img = Image.fromarray(img_rgb)  # Original PIL image
    transformed_img = transform(original_img)  # Resized & transformed image for model inference
    return original_img, transformed_img

def save_image(image_tensor, output_path):
    # Convert tensor (C,H,W) to numpy (H,W,C) and rescale to [0,255]
    image_np = image_tensor.detach().cpu().numpy().transpose(1, 2, 0)
    image_np = np.clip(image_np, 0, 1)
    image_np = (image_np * 255).astype(np.uint8)
    # Convert from RGB to BGR for saving with OpenCV
    image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, image_np)
    print(f"Saved enhanced image to: {output_path}")

def display_side_by_side(original_img, enhanced_img):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(original_img)
    plt.title("Original Image")
    plt.axis("off")
    plt.subplot(1, 2, 2)
    plt.imshow(enhanced_img)
    plt.title("Enhanced Image")
    plt.axis("off")
    plt.show()

# ---------------------------
# Main Inference Function
# ---------------------------
def main(input_path, output_dir, model_checkpoint):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Load the model from the provided checkpoint
    model = DiDCENet().to(device)
    checkpoint = torch.load(model_checkpoint, map_location=device)
    model.load_state_dict(checkpoint)
    model.eval()

    transform = get_inference_transforms()
    os.makedirs(output_dir, exist_ok=True)

    # Determine if input_path is a file or directory
    if os.path.isfile(input_path):
        image_paths = [input_path]
    elif os.path.isdir(input_path):
        image_paths = [os.path.join(input_path, fname) for fname in os.listdir(input_path)
                       if fname.lower().endswith(('.png', '.jpg', '.jpeg'))]
    else:
        raise ValueError("Provided input path is neither a valid file nor a directory.")

    with torch.no_grad():
        for image_path in image_paths:
            # Load image and get both original and model input versions
            original_img, image_tensor = load_image(image_path, transform)
            orig_width, orig_height = original_img.size

            # Add batch dimension and move to device
            image_tensor = image_tensor.unsqueeze(0).to(device)

            # Generate the curve map and perform enhancement
            curve_map = model(image_tensor)
            enhanced_image_tensor = adaptive_light_enhancement(image_tensor.clone(), curve_map)
            
            # Resize the enhanced image back to the original dimensions using bilinear interpolation
            enhanced_image_resized = F.interpolate(enhanced_image_tensor, size=(orig_height, orig_width),
                                                   mode='bilinear', align_corners=False)
            enhanced_image_resized = enhanced_image_resized.squeeze(0)

            # Save the enhanced image
            output_path = os.path.join(output_dir, os.path.basename(image_path))
            save_image(enhanced_image_resized, output_path)

            # Convert tensor to PIL image for display
            enhanced_img_pil = Image.fromarray(
                (np.clip(enhanced_image_resized.cpu().numpy().transpose(1, 2, 0), 0, 1) * 255).astype(np.uint8)
            )
            # Display the original and enhanced images side by side
            display_side_by_side(original_img, enhanced_img_pil)

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Kaggle Inference for DiDCE-Net Image Enhancement")
    # Default paths, modify if necessary
    parser.add_argument("--input_path", type=str,
                        default="/kaggle/input/0002loli/0002.jpeg",
                        help="Path to a single image or directory of images")
    parser.add_argument("--output_dir", type=str,
                        default="/kaggle/working/enhanced_images",
                        help="Directory to save enhanced images")
    parser.add_argument("--checkpoint", type=str,
                        default="/kaggle/input/zerodidcefinalmodel/zero_didce_model.pth",
                        help="Path to the trained model checkpoint")
    
    args = parser.parse_args()
    main(args.input_path, args.output_dir, args.checkpoint)

<ipython-input-3-9929d071ed18>:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_checkpoint, map_location=device)


Saved enhanced image to: /kaggle/working/enhanced_images/0002.jpeg


# Inference for Zero DIDCE Videos

In [5]:
import os
import sys
import cv2
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm
import argparse

# Clear sys.argv so extra arguments from the kernel don't interfere with argparse
sys.argv = ['']

# -------------------------------------------------------------------
# Default Path Configurations for Kaggle
# -------------------------------------------------------------------
MODEL_PATH = "/kaggle/input/zerodidcefinalmodel/zero_didce_model.pth"  # Your model checkpoint path
INPUT_PATH = "/kaggle/input/myvideofordataset/myvideo3.mp4"             # Your input video file
OUTPUT_BASE_DIR = "/kaggle/working/enhanced_output"                     # Directory for enhanced outputs

os.makedirs(OUTPUT_BASE_DIR, exist_ok=True)

# -------------------------------------------------------------------
# Model Definition and Enhancement Function
# -------------------------------------------------------------------
class DiDCENet(nn.Module):
    def __init__(self):
        super(DiDCENet, self).__init__()
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.dropout = nn.Dropout(0.5)
        self.skip_conv = nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        x1 = self.relu(self.conv1(x))
        x2 = self.relu(self.conv2(x1))
        x3 = self.relu(self.conv3(x2))

        # Process inverted input
        inverted_x = 1 - x
        x11 = self.relu(self.conv1(inverted_x))
        x21 = self.relu(self.conv2(x11))
        x31 = self.relu(self.conv3(x21))

        combined_features = torch.cat([x3, x31], dim=1)
        combined_features = self.dropout(combined_features)
        curve_map = torch.tanh(self.skip_conv(combined_features))
        return curve_map

def adaptive_light_enhancement(x_input, curve_map):
    alpha = 0.63  # Target light level
    mean_light_level = torch.mean(x_input).item()
    beta = -0.79 * mean_light_level**2 + 0.81 * mean_light_level + 1.41

    iterations_formula = (
        -25 * mean_light_level + 10 if mean_light_level < 0.1 else
        (17.14 * mean_light_level**2 - 15.14 * mean_light_level + 10 if mean_light_level < 0.45 else
         5.66 * mean_light_level**2 - 2.93 * mean_light_level + 7.2)
    )
    iterations = int(iterations_formula)
    enhanced_image = x_input.clone()
    for _ in range(iterations):
        enhanced_image = enhanced_image + curve_map * (enhanced_image**2 - enhanced_image) * \
                         ((alpha - torch.mean(enhanced_image).item()) / (beta - torch.mean(enhanced_image).item()))
    return enhanced_image

# -------------------------------------------------------------------
# Image Processing Functions (if needed)
# -------------------------------------------------------------------
def get_inference_transforms():
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    return transform

def load_image(image_path, transform):
    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        raise ValueError(f"Unable to read image at {image_path}")
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    original_img = Image.fromarray(img_rgb)
    transformed_img = transform(original_img)
    return original_img, transformed_img

def save_image(image_tensor, output_path):
    image_np = image_tensor.detach().cpu().numpy().transpose(1, 2, 0)
    image_np = np.clip(image_np, 0, 1)
    image_np = (image_np * 255).astype(np.uint8)
    image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, image_np)
    print(f"Saved enhanced image to: {output_path}")

def display_side_by_side(original_img, enhanced_img):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(original_img)
    plt.title("Original Image")
    plt.axis("off")
    plt.subplot(1, 2, 2)
    plt.imshow(enhanced_img)
    plt.title("Enhanced Image")
    plt.axis("off")
    plt.show()

# -------------------------------------------------------------------
# Video Processing Functions
# -------------------------------------------------------------------
def process_video(input_path, output_path, model, device, transform):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        raise ValueError(f"Unable to open video file: {input_path}")
    
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    progress_bar = tqdm(total=total_frames, desc="Processing video frames", unit="frame")
    
    with torch.no_grad():
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_image = Image.fromarray(frame_rgb)
            image_tensor = transform(pil_image).unsqueeze(0).to(device)

            curve_map = model(image_tensor)
            enhanced_image_tensor = adaptive_light_enhancement(image_tensor.clone(), curve_map)
            enhanced_image_resized = F.interpolate(enhanced_image_tensor, 
                                                   size=(height, width),
                                                   mode='bilinear',
                                                   align_corners=False)
            enhanced_frame = enhanced_image_resized.squeeze(0).cpu().numpy().transpose(1, 2, 0)
            enhanced_frame = np.clip(enhanced_frame * 255., 0, 255).astype(np.uint8)
            # Convert RGB to BGR
            enhanced_frame_bgr = cv2.cvtColor(enhanced_frame, cv2.COLOR_RGB2BGR)
            out.write(enhanced_frame_bgr)

            progress_bar.update(1)
            
    progress_bar.close()
    cap.release()
    out.release()
    print(f"Video processing complete. Saved to: {output_path}")

# -------------------------------------------------------------------
# Main Processing Function
# -------------------------------------------------------------------
def main(input_path, output_dir, model_checkpoint, mode='video'):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    # Load model from checkpoint
    model = DiDCENet().to(device)
    checkpoint = torch.load(model_checkpoint, map_location=device)
    model.load_state_dict(checkpoint)
    model.eval()

    transform = get_inference_transforms()
    os.makedirs(output_dir, exist_ok=True)

    if mode in ['both', 'video']:
        print("Processing video...")
        if os.path.isfile(input_path):
            video_paths = [input_path]
        else:
            video_paths = [os.path.join(input_path, fname) for fname in os.listdir(input_path)
                           if fname.lower().endswith(('.mp4', '.avi', '.mov'))]
        
        for video_path in video_paths:
            out_path = os.path.join(output_dir, 'enhanced_' + os.path.basename(video_path))
            process_video(video_path, out_path, model, device, transform)

    if mode in ['both', 'image']:
        print("Processing images...")
        # Code for image processing (if desired) goes here.
        pass

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Kaggle Image and Video Enhancement using DiDCE-Net")
    parser.add_argument("--input_path", type=str, default=INPUT_PATH,
                        help="Path to input image/video file or directory")
    parser.add_argument("--output_dir", type=str, default=OUTPUT_BASE_DIR,
                        help="Directory to save enhanced media")
    parser.add_argument("--checkpoint", type=str, default=MODEL_PATH,
                        help="Path to the trained model checkpoint")
    parser.add_argument("--mode", type=str, default="video", choices=['image', 'video', 'both'],
                        help="Processing mode: 'image', 'video', or 'both'")
    
    args = parser.parse_args()
    main(args.input_path, args.output_dir, args.checkpoint, args.mode)

<ipython-input-5-73d4508c6782>:167: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_checkpoint, map_location=device)


Using device: cuda
Processing video...


Processing video frames: 100%|██████████| 149/149 [00:13<00:00, 11.29frame/s]

Video processing complete. Saved to: /kaggle/working/enhanced_output/enhanced_myvideo3.mp4
